# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 1.0 MB 8.6 MB/s 
     |████████████████████████████████| 15.4 MB 84 kB/s 
     |████████████████████████████████| 6.3 MB 2.4 MB/s 


In [2]:
# Reading in all data for cleaning and preprocessing #

test_data = pd.read_excel("/content/drive/MyDrive/Wells Fargo Project/b765dc3d8076-testset_for_participants.xlsx")
train_data = pd.read_excel("/content/drive/MyDrive/Wells Fargo Project/b765dc3d8076-trainset+(1).xlsx")
census_data = pd.read_csv("/content/drive/MyDrive/Wells Fargo Project/ACSST5Y2019.S1901_data_with_overlays_2021-09-09T142232.csv")
df = gpd.read_file('/content/drive/MyDrive/Wells Fargo Project/tl_2020_us_zcta520/tl_2020_us_zcta520.shp')
fraud_rates = pd.read_csv("/content/drive/MyDrive/Wells Fargo Project/Fraud.txt",sep = '\t', header = None).rename(columns = {0:'overall_rank',1:'state',2:'fraud_score',
                                                                                                                              3:'id_theft_rank', 4:'fraud_rank',5:'policy rank'})

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,34,35,66,67,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Cleaning ACS 5-year 2019 data, selecting the desired rows from the dataset and merging them to the testing and training dataset by zipcode #

zip_income = census_data[['NAME','S1901_C01_012E','S1901_C01_013E']][1:].rename(columns = {'NAME':'zipcode','S1901_C01_012E':'median_income','S1901_C01_013E':'mean_income'})
#zip_cleaning = zip_income.copy()
columns = list(zip_income.columns)
series_list = [zip_income.median_income,zip_income.mean_income]
cleaned_dict = {}
mender = []
i = 0

# Extracting zipcode values from series object in zip_income dataframe and appending to an empty dictionary #

zip_income['zipcode'] = zip_income.zipcode.str.extract(r'\w+ (\d+)')
cleaned_dict['zipcode'] = list(zip_income.zipcode)

# Handling string value objects in dataset with mixed (character and numerical string values) the goal is to fill character strings with 0 and turn the numerical strings into integers

for series in series_list: # isolates a single series object in list of series
  i += 1
  for element in range(1,len(series) + 1): # isolates every element within the selected series
    try: # Will conduct the code in the next two lines below until a ValueError occurs
      make_int = int(series[element])
      mender.append(make_int)
    except ValueError:
      series[element] = np.nan
      mender.append(series[element])
  cleaned_dict[columns[i]] = mender # Names dictionary key based on iteratively selected column name uses mender list as value to form a new key:value pair in dictionary
  mender = [] # Clears list for use in iteration of for loop to the next list

zip_income = pd.DataFrame(cleaned_dict).fillna(0)
zip_income = zip_income.astype({'zipcode':'int64'})

In [4]:
zip_income.to_csv("/content/drive/MyDrive/Wells Fargo Project/zip_income.csv")

In [5]:
# Handling NaNs in dataframes based on column dtype replacing them with either a zero or missing #

series_dict = {}

for column in train_data.columns:
  Series = train_data[column]
  get_dtype = Series.dtype
  get_dtype = '{get_dtype}'.format(get_dtype = get_dtype)
  if (get_dtype == 'float64'):
    Series = Series.fillna(0.0)
    series_dict['{column}'.format(column = column)] = list(Series)
  elif (get_dtype == 'int64'):
    Series = Series.fillna(0)
    series_dict['{column}'.format(column = column)] = list(Series)  
  elif (get_dtype == 'object'):
    Series = Series.fillna('missing') 
    series_dict['{column}'.format(column = column)] = list(Series)
  else:
    series_dict['{column}'.format(column = column)] = list(Series)

train_data = pd.DataFrame(series_dict)

# Repeating above code for test dataset #

series_dict1 = {}

for column in test_data.columns:
  Series = test_data[column]
  get_dtype = Series.dtype
  get_dtype = '{get_dtype}'.format(get_dtype = get_dtype)
  if (get_dtype == 'float64'):
    Series = Series.fillna(0.0)
    series_dict1['{column}'.format(column = column)] = list(Series)
  elif (get_dtype == 'int64'):
    Series = Series.fillna(0)
    series_dict1['{column}'.format(column = column)] = list(Series)  
  elif (get_dtype == 'object'):
    Series = Series.fillna('missing') 
    series_dict1['{column}'.format(column = column)] = list(Series)
  else:
    series_dict1['{column}'.format(column = column)] = list(Series)

test_data = pd.DataFrame(series_dict1)





In [6]:
# identifies all zipcodes in the ACS 5-year 2019 dataset which are also in the training dataset it reorganizes the dataframe and a merge step is performed to combine the two datasets

all_zips = list(zip_income.zipcode)
zip_income_cleaned = pd.DataFrame()

for zip in all_zips:
  if zip in list(train_data.CUST_ZIP):
    in_zip = zip_income[zip_income.zipcode == zip]
    zip_income_cleaned = pd.concat([zip_income_cleaned,in_zip], axis = 0)


train_data = train_data.merge(zip_income_cleaned, left_on = 'CUST_ZIP', right_on = 'zipcode')
zip_income_cleaned = pd.DataFrame()

# Repeat of the previous steps for the testing data

for zip in all_zips:
  if zip in list(test_data.CUST_ZIP):
    in_zip = zip_income[zip_income.zipcode == zip]
    zip_income_cleaned = pd.concat([zip_income_cleaned,in_zip], axis = 0)

test_data = test_data.merge(zip_income_cleaned, left_on = 'CUST_ZIP', right_on = 'zipcode')

In [7]:
# identifies states present in fraud_rates dataset also present in train and test datasets and merges based on the results of the initial state matching #

fraud_rates_cleaned = pd.DataFrame()

for states in list(fraud_rates.state):
  if states in list(train_data.CUST_STATE):
    in_state = fraud_rates[fraud_rates.state == states]
    fraud_rates_cleaned = pd.concat([fraud_rates_cleaned, in_state], axis = 0)

train_data = train_data.merge(fraud_rates_cleaned, left_on = 'CUST_STATE', right_on = 'state')

fraud_rates_cleaned = pd.DataFrame()

# Repeating steps from code above #

for states in list(fraud_rates.state):
  if states in list(test_data.CUST_STATE):
    in_state = fraud_rates[fraud_rates.state == states]
    fraud_rates_cleaned = pd.concat([fraud_rates_cleaned, in_state], axis = 0)

test_data = test_data.merge(fraud_rates_cleaned, left_on = 'CUST_STATE', right_on = 'state')




In [8]:
# Dropping rows from both the training and testing datasets #

train_data = train_data.drop(columns = ['ACTN_CD','ACTN_INTNL_TXT','TRAN_TYPE_CD','ACTVY_DT','zipcode'])
test_data = test_data.drop(columns = ['ACTN_CD','ACTN_INTNL_TXT','TRAN_TYPE_CD','ACTVY_DT','zipcode'])

In [9]:
# Categorizing CUST_AGE column #

def age_categorize(dataset):

  age_cats = []
  for age in list(dataset.CUST_AGE.astype('int64')):
    if (age >= 0) & (age <= 29):
      age_cats.append('Young Adult')
    elif (age >= 30) & (age <= 59):
      age_cats.append('Adult')
    elif (age >= 60):
      age_cats.append('Senior')
  
  dataset['CUST_AGE'] = age_cats

  return dataset

def get_times(dataset,column, year_since = 0):

    dataset['year'] = dataset[column].str.extract(r'\d{,100}/\d{,100}/(\d{,100}) \d{1,2}:\d{1,2}:\d{1,2}').fillna(9999)
    t_list = []
    int_list = []

    if year_since == 0:

        dataset['month'] = dataset[column].str.extract(r'(\d{,100})/\d{,100}/\d{,100} \d{1,2}:\d{1,2}:\d{1,2}')
        dataset['tod'] = dataset[column].str.extract(r'\d{,100}/\d{,100}/\d{,100} (\d{1,2}):\d{1,2}:\d{1,2}')

        for value in range(0,len(dataset.tod)):   
            try:
                make_int = int(dataset.tod[value])
                int_list.append(make_int)
            except ValueError:
                int_list.append(np.nan)

        for hour in int_list:
            if (hour >= 0) & (hour <= 5):
                t_list.append('Early Morning')
            elif (hour >=6) & (hour <= 11):
                t_list.append('Morning')
            elif (hour >= 12) & (hour <= 17):
                t_list.append('Afternoon')
            elif (hour >= 18) & (hour <= 23):
                t_list.append('Evening')
            else:
                t_list.append('Unknown')
        

        dataset[column] = t_list
        dataset = dataset.drop(columns = ['tod'])


    elif year_since == 1:

        dataset['year'] = dataset.year.astype('int64')
        dataset['years_since'] = 2021 - dataset['year']
        dataset = dataset.drop(columns = ['year'])

        for years in list(dataset['years_since']):
            if (years >= 0) & (years <=1):
                t_list.append('1 year or Less')
            elif (years > 1) & (years <= 3):
                t_list.append('2-3 years')
            elif (years > 3) & (years <= 4):
                t_list.append('3-4 years')
            elif (years >= 5):
                t_list.append('5+ years')
            elif (years < 0):
                t_list.append('Unknown')
    
        dataset[column] = t_list
        dataset = dataset.drop(columns = ['years_since'])

    return dataset



    





In [10]:
# Modifying train_data
train_data['AGE'] = train_data.CUST_AGE
train_data = age_categorize(train_data)
train_data = get_times(train_data,'PWD_UPDT_TS',year_since = 1)
train_data = get_times(train_data,'PH_NUM_UPDT_TS',year_since = 1)
train_data = get_times(train_data,'TRAN_TS')

In [11]:
test_data['AGE'] = test_data.CUST_AGE
test_data = age_categorize(test_data)
test_data = get_times(test_data,'PWD_UPDT_TS',year_since = 1)
test_data = get_times(test_data,'PH_NUM_UPDT_TS',year_since = 1)
test_data = get_times(test_data,'TRAN_TS')

In [12]:
train_data1 = train_data[train_data.FRAUD_NONFRAUD == 'Fraud']
fraud_hits = train_data1.groupby(['CUST_ZIP'])[['CUST_ZIP']].count().rename({'CUST_ZIP':'fraud_hits'}, axis = 'columns').reset_index().astype({'CUST_ZIP':'int64'})

In [13]:
med_income = list()
zip_list = list()

for zip in list(df.ZCTA5CE20.astype('int64')):
  zip_list.append(zip)
  if zip in list(train_data.CUST_ZIP):
    get_income = list(train_data[train_data.CUST_ZIP == zip]['median_income'])
    med_income.append(get_income[0])
  elif zip not in list(train_data.CUST_ZIP):
    med_income.append(0)

In [14]:
f_hits = list()
zip_list = list()

for zip in list(df.ZCTA5CE20):
  zip_list.append(zip)
  zip = int(zip)
  if zip in list(fraud_hits.CUST_ZIP):
    get_hits = list(fraud_hits[fraud_hits.CUST_ZIP == zip].fraud_hits)[0]
    f_hits.append(get_hits)
  elif zip not in list(fraud_hits.CUST_ZIP):
    f_hits.append(0)

In [15]:
#zip_income = zip_income.drop(['Unnamed: 0','mean_income'],axis = 1)
zip_income = zip_income.astype({'zipcode':'int64'})

In [16]:
data = {'ZIP':zip_list,'CUST_ZIP_median_income':med_income,'fraud_hits':f_hits}
geo_df = df.astype({'ZCTA5CE20':'int64'}).merge(pd.DataFrame(data).astype({'ZIP':'int64'}),left_on = 'ZCTA5CE20',right_on = 'ZIP').drop('ZIP',axis = 1)
geo_df = geo_df.merge(zip_income,left_on = 'ZCTA5CE20',right_on = 'zipcode').drop('zipcode',axis = 1)

In [17]:
alert_zips = train_data.groupby(['CUST_ZIP'])[['CUST_ZIP']].count().rename({'CUST_ZIP':'alert_counts'}, axis = 'columns').reset_index().astype({'CUST_ZIP':'int64'})
alert_ratio1 = fraud_hits.merge(alert_zips,left_on = 'CUST_ZIP', right_on = 'CUST_ZIP').sort_values('alert_counts',ascending = False)

In [18]:
zip_list = list()
f_list = list()
for zip in list(train_data.CUST_ZIP):
  zip_list.append(zip)
  if zip in list(fraud_hits.CUST_ZIP):
    get_fraud = list(fraud_hits[fraud_hits.CUST_ZIP == zip]['fraud_hits'])[0]
    f_list.append(get_fraud)
  elif zip not in list(fraud_hits.CUST_ZIP):
    f_list.append(0)

In [19]:
alert_ratio = pd.DataFrame({'CUST_ZIP':zip_list,'fraud_hits':f_list}).merge(alert_zips,left_on = 'CUST_ZIP',right_on = 'CUST_ZIP')
alert_ratio['fraud_alert_ratio'] = alert_ratio.fraud_hits/alert_ratio.alert_counts
alert_ratio

,CUST_ZIP,fraud_hits,alert_counts,fraud_alert_ratio
0,89002,1,3,0.333333
1,89002,1,3,0.333333
2,89002,1,3,0.333333
3,89822,0,5,0.000000
4,89822,0,5,0.000000
...,...,...,...,...
13685,25430,3,3,1.000000
13686,25430,3,3,1.000000
13687,25430,3,3,1.000000
13688,26330,0,1,0.000000


In [20]:
train_data = pd.concat([train_data,alert_ratio[['fraud_hits','alert_counts','fraud_alert_ratio']]],axis = 1)


In [21]:
zip_list = list()
fraud_rate = list()
alert = list()
f_list = list()
for zip in list(test_data.CUST_ZIP):
  zip_list.append(zip)
  if zip in list(alert_ratio.CUST_ZIP):
    get_rate = list(alert_ratio[alert_ratio.CUST_ZIP == zip]['fraud_alert_ratio'])[0]
    get_hits = list(alert_ratio[alert_ratio.CUST_ZIP == zip]['fraud_hits'])[0]
    get_counts = list(alert_ratio[alert_ratio.CUST_ZIP == zip]['alert_counts'])[0]
    fraud_rate.append(get_rate)
    alert.append(get_counts)
    f_list.append(get_hits)
  elif zip not in list(alert_ratio.CUST_ZIP):
    fraud_rate.append(0.0) 
    alert.append(0.0)
    f_list.append(0.0)

In [22]:
test_data = pd.concat([test_data,pd.Series(f_list).rename('fraud_hits'),pd.Series(alert).rename('alert_counts'),
           pd.Series(fraud_rate).rename('fraud_alert_ratio')],axis = 1)


In [23]:
 train_data.to_csv("/content/drive/MyDrive/Wells Fargo Project/train_data.csv")

In [24]:
test_data.to_csv("/content/drive/MyDrive/Wells Fargo Project/test_data.csv")

In [25]:
geo_df.to_file("/content/drive/MyDrive/Wells Fargo Project/geo_df/geo_df.shp")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [26]:
alert_ratio1.to_csv("/content/drive/MyDrive/Wells Fargo Project/alert_ratio1.csv")
alert_ratio.to_csv("/content/drive/MyDrive/Wells Fargo Project/alert_ratio.csv")
